# Exercise-2
Write an implementation of a rainbow table attack against the  
Microsoft LAN Manager password-hashing scheme.

In [ ]:
from lmdes import lmdes, bitarray2str
from rainbow_lm import RainbowTable
from bitarray import bitarray

In [ ]:
bit_array = bitarray()
for i in range(7 * 8):
    bit_array.append(0)
print(bit_array)
pwd_zero = bitarray2str(bit_array)

In [ ]:
word = "17437184211106"
target = lmdes(word).generate_hash()
h1, h2 = target[:16], target[16:]
print("Target hash:", target)
print("Hash h1:", h1)
print("Hash h2:", h2)

In [ ]:
numbers = '0123456789'
rain = RainbowTable(columns=10000, chars=numbers,rows=3048)

In [ ]:
rain.__repr__
rain.writeToFile("rain_num_R=3048_C=10000.txt")

In [ ]:
# rain = RainbowTable()
# rain.readFromFile("rain_num_R=20000_C=200.txt")
# rain.__repr__

In [ ]:
pwd1 = rain.crackHash(h1)

In [ ]:
# if h2 == 'aad3b435b51404ee':
#     pwd2 = pwd_zero
# else:  
#     print("Password longer than 7 characters")
#     pwd2 = rain.crackHash(h2)

In [ ]:
print("Cracked password (1): %r" % (pwd1))
# print("Cracked password (2): %r" % (pwd2))

In [ ]:
# rain.writeToFile("rain_num.txt")

# Exercise-3
Slide attack on EASY1 with 20 rounds

In [1]:
import random
from easy1 import encrypt, decrypt, mux, demux, sbox, pbox

In [2]:
####################################################
# Given parameters
####################################################
rounds = 20
n_block, n_key = 36, 18
n_trials = 2 ** (n_block // 2)
pairs_file = "plain-cipher.txt"
print("Number of PT-CT pairs to be generated:", n_trials)
# secret_key = 1
secret_key = random.getrandbits(n_key)
print("Secret key:", secret_key)

Number of PT-CT pairs to be generated: 262144
Secret key: 184895


In [3]:
####################################################
# Encrypt and generate PT-CT pairs
####################################################
plain_cipher_pairs = []
for i in range(n_trials):
    plain = random.getrandbits(n_block)
    cipher = encrypt(plain, rounds, secret_key)
    plain_cipher_pairs.append((plain, cipher))

In [4]:
####################################################
# Write PT-CT pairs to a file
####################################################
f = open(pairs_file, 'w')
for pt, ct in plain_cipher_pairs:
    f.write(" ".join([str(pt), str(ct)]))
    f.write("\n")
f.close()

In [5]:
####################################################
# Functions for the attack
####################################################
def f_spn(p):
    u = []
    # Run through the s-boxes
    for x in demux(p):
        u.append(sbox(x))
    # Run through the p-box and return
    return pbox(mux(u))

def attack(PC, rounds):
    for inx, pair in enumerate(PC):
        print('Running pair# %d out of %d pairs' % (inx+1, len(PC)))
        p, c = pair
        fp, fc = f_spn(p), f_spn(c)
        PC_copy = PC.copy()
        PC_copy.remove(pair)
        for q, d in PC_copy:
            if fp ^ q == fc ^ d:
                k = (fp ^ q) & 0x3ffff
                print('Checking for collision with key =', k)
                if encrypt(p, rounds, k) == c:
                    print('Found key!')
                    return k
                else:
                    print('False positive!')
    return None

In [6]:
####################################################
# Read the file with PT-CT pairs
####################################################
PC = []
f = open(pairs_file, "r")
line = f.readline()
while line != '':
    pt, ct = line.strip().split(sep=" ", maxsplit=1)
    PC.append((int(pt), int(ct)))
    line = f.readline()
f.close()

In [ ]:
key = attack(PC, rounds)
print("Secret key:", key)